In [18]:
from svm_classifier import getClassifierAndVectorizer
import numpy as np

In [13]:
clf, vectorizer = getClassifierAndVectorizer()

imported correctly


In [21]:
x_test = vectorizer.transform(data_to_predict)


AttributeError: 'numpy.ndarray' object has no attribute 'lower'